# Load in the .ply file as a dataframe.

In [2]:
from plyfile import PlyData
import pandas as pd
from tqdm.notebook import tqdm

# Load the .ply file
ply = PlyData.read("bathtub.ply")

# Extract vertex data (assuming Gaussian splats are stored under 'vertex')
vertex_data = ply['vertex'].data  # Structured NumPy array

# Convert structured array to Pandas DataFrame
df = pd.DataFrame(vertex_data)

# Print first few rows to verify
df.head()

NUM_GAUSSIANS = df.shape[0]
NUM_GAUSSIANS

17103

# Extract covariance matrix

In [3]:
import numpy as np
import pandas as pd



def quaternion_to_rotation_matrix(w, x, y, z):
    """ Convert a quaternion (w, x, y, z) into a 3×3 rotation matrix. """
    return np.array([
        [1 - 2 * (y**2 + z**2), 2 * (x*y - w*z), 2 * (x*z + w*y)],
        [2 * (x*y + w*z), 1 - 2 * (x**2 + z**2), 2 * (y*z - w*x)],
        [2 * (x*z - w*y), 2 * (y*z + w*x), 1 - 2 * (x**2 + y**2)]
    ])



p_bar = tqdm(range(NUM_GAUSSIANS))
curr = 0
def compute_covariance_matrix(row):

    """ Compute the covariance matrix for a given row with quaternion and scale values. """
    # Extract quaternion and scale values
    w, x, y, z = row['rot_0'], row['rot_1'], row['rot_2'], row['rot_3']
    scale_0, scale_1, scale_2 = row['scale_0'], row['scale_1'], row['scale_2']

    # Compute rotation matrix
    R = quaternion_to_rotation_matrix(w, x, y, z)

    # Compute scale diagonal matrix (square of scale values)
    D = np.diag([scale_0**2, scale_1**2, scale_2**2])

    # Compute covariance matrix: Σ = R D R^T
    cov_matrix = R @ D @ R.T

    # Progress bar stuff
    global curr
    if curr % 100 == 0:
        p_bar.update(100)
    curr += 1
    p_bar.refresh()
    return cov_matrix

# Compute covariance matrices for each row
df['cov_matrix'] = df.apply(compute_covariance_matrix, axis=1)

# Display the first few results
df['cov_matrix']

  0%|          | 0/17103 [00:00<?, ?it/s]

0        [[14.2551565, 0.31448466, -0.5548387], [0.3144...
1        [[11.827917, -0.45735186, -0.41885278], [-0.45...
2        [[15.732886, -0.7379525, -0.5710099], [-0.7379...
3        [[18.020498, 0.24299759, -1.0292158], [0.24299...
4        [[15.156092, -1.3880357, -0.4204048], [-1.3880...
                               ...                        
17098    [[26.211916, 0.2540651, 1.714622], [0.25406456...
17099    [[44.5594, -4.6726885, 5.749929], [-4.6726885,...
17100    [[17.984083, -4.4060597, 2.3991559], [-4.40605...
17101    [[23.549572, -9.974669, -1.9258583], [-9.97466...
17102    [[49.3649, -6.307815, 5.9685583], [-6.3078146,...
Name: cov_matrix, Length: 17103, dtype: object

# Calculate eigenvectors, eigenvalues and subsequently the normal for each gaussian

In [4]:
def quaternion_to_rotation_matrix(w, x, y, z):
    """Convert a quaternion (w, x, y, z) into a 3×3 rotation matrix.
    
    Args:
        w: Scalar/real component of quaternion
        x, y, z: Vector/imaginary components of quaternion
        
    Returns:
        3x3 rotation matrix as numpy array
    """
    return np.array([
        [1 - 2*y*y - 2*z*z,     2*x*y - 2*w*z,     2*x*z + 2*w*y],
        [    2*x*y + 2*w*z, 1 - 2*x*x - 2*z*z,     2*y*z - 2*w*x],
        [    2*x*z - 2*w*y,     2*y*z + 2*w*x, 1 - 2*x*x - 2*y*y]
    ])
    

p_bar = tqdm(range(NUM_GAUSSIANS))
curr = 0

def compute_normal_from_rotation(row):
    w, x, y, z = row['rot_0'], row['rot_1'], row['rot_2'], row['rot_3']
    R = quaternion_to_rotation_matrix(w, x, y, z)
    
    # Extract the normal (assuming it aligns with z-axis)
    # R[:, 2] gives us the third column of R
    normal = R[:, 2]  # or R[2, :] depending on your convention
    global curr
    if curr % 1000 == 0:
        p_bar.update(1000)
        p_bar.refresh()
    curr += 1
    
    return normal[0], normal[1], normal[2]

df['n_x'], df['n_y'], df['n_z'] = zip(*df.apply(compute_normal_from_rotation, axis=1))
df_normals = df[['x', 'y', 'z', 'n_x', 'n_y', 'n_z']]

  0%|          | 0/17103 [00:00<?, ?it/s]

In [5]:
df_normals.to_csv('normals.csv')